In [3]:
import tensorflow as tf

In [4]:
shakespeare_url = "https://homl.info/shakespeare"
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [5]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [7]:
# character encoding
text_vec_layer = tf.keras.layers.TextVectorization(split="character", standardize="lower")
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]
encoded

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([21,  7, 10, ..., 22, 28, 12])>

In [8]:
# delete tokens 0 and 1 (filled, unknown)
encoded -= 2
# amount of distinct letters
n_tokens = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)

In [9]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    # split data to windows (+1 length for target)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    # dataset with elements of length: length+1
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)
    ds = ds.batch(batch_size)
    # mapping input and target data
    # for example text 'abcde' as an input
    # 'bcdef' as a target
    return ds.map(lambda window: (window[:,:-1], window[:, 1:])).prefetch(1)

In [10]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True, seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

Model rnn

In [11]:
model = tf.keras.Sequential([
    # coding letters to numbers
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
# save the best model according to validation set accuracy
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "shakespeare_model.keras", monitor="val_accuracy", save_best_only=True
)

In [ ]:
history = model.fit(train_set, validation_data=valid_set, epochs=10, callbacks=[model_ckpt])

In [12]:
# model with text preprocessing
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),
    model
])

In [ ]:
def next_char(text, temperature=1):
    # probabilities for each letter
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    # drawing a letter
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [ ]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text+=next_char(text, temperature)
    return text

Stateful rnn

In [13]:
def to_stateful_rnn_dataset(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=length, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(length + 1)).batch(1)
    return ds.map(lambda window: (window[:,:-1], window[:, 1:])).prefetch(1)

In [15]:
stateful_train_set = to_stateful_rnn_dataset(encoded[:1_000_000], length)
stateful_valid_set = to_stateful_rnn_dataset(encoded[1_000_000:1_060_000], length)
stateful_test_set = to_stateful_rnn_dataset(encoded[1_060_000:], length)